In [1]:
import math

def calc_time_julian_cent(jd):
    T = (jd - 2451545.0) / 36525.0
    return T

def calc_jd_from_julian_cent(t):
    JD = t * 36525.0 + 2451545.0
    return JD

def is_leap_year(yr):
    return (yr % 4 == 0 and yr % 100 != 0) or yr % 400 == 0

def calc_date_from_jd(jd):
    z = int(jd + 0.5)
    f = (jd + 0.5) - z
    if z < 2299161:
        A = z
    else:
        alpha = int((z - 1867216.25) / 36524.25)
        A = z + 1 + alpha - int(alpha / 4)
    B = A + 1524
    C = int((B - 122.1) / 365.25)
    D = int(365.25 * C)
    E = int((B - D) / 30.6001)
    day = B - D - int(30.6001 * E) + f
    month = E - 1 if E < 14 else E - 13
    year = C - 4716 if month > 2 else C - 4715
    return {"year": year, "month": month, "day": day}

def calc_doy_from_jd(jd):
    date = calc_date_from_jd(jd)
    k = 1 if is_leap_year(date["year"]) else 2
    doy = int((275 * date["month"]) / 9) - k * int((date["month"] + 9) / 12) + date["day"] - 30
    return doy

def rad_to_deg(angle_rad):
    return (180.0 * angle_rad / math.pi)

def deg_to_rad(angle_deg):
    return (math.pi * angle_deg / 180.0)

def calc_geom_mean_long_sun(t):
    L0 = 280.46646 + t * (36000.76983 + t * (0.0003032))
    while L0 > 360.0:
        L0 -= 360.0
    while L0 < 0.0:
        L0 += 360.0
    return L0  # in degrees

def calc_geom_mean_anomaly_sun(t):
    M = 357.52911 + t * (35999.05029 - 0.0001537 * t)
    return M  # in degrees

def calc_eccentricity_earth_orbit(t):
    e = 0.016708634 - t * (0.000042037 + 0.0000001267 * t)
    return e  # unitless

def calc_sun_eq_of_center(t):
    m = calc_geom_mean_anomaly_sun(t)
    mrad = deg_to_rad(m)
    sinm = math.sin(mrad)
    sin2m = math.sin(mrad + mrad)
    sin3m = math.sin(mrad + mrad + mrad)
    C = sinm * (1.914602 - t * (0.004817 + 0.000014 * t)) + sin2m * (0.019993 - 0.000101 * t) + sin3m * 0.000289
    return C  # in degrees

def calc_sun_true_long(t):
    l0 = calc_geom_mean_long_sun(t)
    c = calc_sun_eq_of_center(t)
    O = l0 + c
    return O  # in degrees

def calc_sun_true_anomaly(t):
    m = calc_geom_mean_anomaly_sun(t)
    c = calc_sun_eq_of_center(t)
    v = m + c
    return v  # in degrees

def calc_sun_rad_vector(t):
    v = calc_sun_true_anomaly(t)
    e = calc_eccentricity_earth_orbit(t)
    R = (1.000001018 * (1 - e * e)) / (1 + e * math.cos(deg_to_rad(v)))
    return R  # in AUs

def calc_sun_apparent_long(t):
    o = calc_sun_true_long(t)
    omega = 125.04 - 1934.136 * t
    lambda_ = o - 0.00569 - 0.00478 * math.sin(deg_to_rad(omega))
    return lambda_  # in degrees

def calc_mean_obliquity_of_ecliptic(t):
    seconds = 21.448 - t * (46.8150 + t * (0.00059 - t * (0.001813)))
    e0 = 23.0 + (26.0 + (seconds / 60.0)) / 60.0
    return e0  # in degrees

def calc_obliquity_correction(t):
    e0 = calc_mean_obliquity_of_ecliptic(t)
    omega = 125.04 - 1934.136 * t
    e = e0 + 0.00256 * math.cos(deg_to_rad(omega))
    return e  # in degrees

def calc_sun_rt_ascension(t):
    e = calc_obliquity_correction(t)
    lambda_ = calc_sun_apparent_long(t)
    tananum = (math.cos(deg_to_rad(e)) * math.sin(deg_to_rad(lambda_)))
    tanadenom = (math.cos(deg_to_rad(lambda_)))
    alpha = rad_to_deg(math.atan2(tananum, tanadenom))
    return alpha  # in degrees

def calc_sun_declination(t):
    e = calc_obliquity_correction(t)
    lambda_ = calc_sun_apparent_long(t)
    sint = math.sin(deg_to_rad(e)) * math.sin(deg_to_rad(lambda_))
    theta = rad_to_deg(math.asin(sint))
    return theta  # in degrees

def calc_equation_of_time(t):
    epsilon = calc_obliquity_correction(t)
    l0 = calc_geom_mean_long_sun(t)
    e = calc_eccentricity_earth_orbit(t)
    m = calc_geom_mean_anomaly_sun(t)
    y = math.tan(deg_to_rad(epsilon) / 2.0)
    y *= y
    sin2l0 = math.sin(2.0 * deg_to_rad(l0))
    sinm = math.sin(deg_to_rad(m))
    cos2l0 = math.cos(2.0 * deg_to_rad(l0))
    sin4l0 = math.sin(4.0 * deg_to_rad(l0))
    sin2m = math.sin(2.0 * deg_to_rad(m))
    Etime = y * sin2l0 - 2.0 * e * sinm + 4.0 * e * y * sinm * cos2l0 - 0.5 * y * y * sin4l0 - 1.25 * e * e * sin2m
    return rad_to_deg(Etime) * 4.0  # in minutes of time

def calc_hour_angle_sunrise(lat, solar_dec):
    lat_rad = deg_to_rad(lat)
    sd_rad = deg_to_rad(solar_dec)
    HAarg = (math.cos(deg_to_rad(90.833)) / (math.cos(lat_rad) * math.cos(sd_rad)) - math.tan(lat_rad) * math.tan(sd_rad))
    HA = math.acos(HAarg)
    return HA  # in radians (for sunset, use -HA)

def is_number(input_val):
    try:
        float(input_val)
        return True
    except ValueError:
        return False

def get_jd(year, month, day):
    if month <= 2:
        year -= 1
        month += 12
    A = year // 100
    B = 2 - A + A // 4
    JD = int(365.25 * (year + 4716)) + int(30.6001 * (month + 1)) + day + B - 1524.5
    return JD

def calc_refraction(elev):
    if elev > 85.0:
        correction = 0.0
    else:
        te = math.tan(deg_to_rad(elev))
        if elev > 5.0:
            correction = 58.1 / te - 0.07 / (te**3) + 0.000086 / (te**5)
        elif elev > -0.575:
            correction = 1735.0 + elev * (-518.2 + elev * (103.4 + elev * (-12.79 + elev * 0.711)))
        else:
            correction = -20.774 / te
        correction = correction / 3600.0
    return correction

def calc_az_el(T, local_time, latitude, longitude, zone):
    eq_time = calc_equation_of_time(T)
    theta = calc_sun_declination(T)
    solar_time_fix = eq_time + 4.0 * longitude - 60.0 * zone
    earth_rad_vec = calc_sun_rad_vector(T)
    true_solar_time = local_time + solar_time_fix
    while true_solar_time > 1440:
        true_solar_time -= 1440
    hour_angle = true_solar_time / 4.0 - 180.0
    if hour_angle < -180:
        hour_angle += 360.0
    ha_rad = deg_to_rad(hour_angle)
    csz = math.sin(deg_to_rad(latitude)) * math.sin(deg_to_rad(theta)) + math.cos(deg_to_rad(latitude)) * math.cos(deg_to_rad(theta)) * math.cos(ha_rad)
    csz = min(1.0, max(-1.0, csz))
    zenith = rad_to_deg(math.acos(csz))
    az_denom = (math.cos(deg_to_rad(latitude)) * math.sin(deg_to_rad(zenith)))
    if abs(az_denom) > 0.001:
        az_rad = ((math.sin(deg_to_rad(latitude)) * math.cos(deg_to_rad(zenith))) - math.sin(deg_to_rad(theta))) / az_denom
        az_rad = min(1.0, max(-1.0, az_rad))
        azimuth = 180.0 - rad_to_deg(math.acos(az_rad))
        if hour_angle > 0.0:
            azimuth = -azimuth
    else:
        azimuth = 180.0 if latitude > 0.0 else 0.0
    if azimuth < 0.0:
        azimuth += 360.0
    exoatm_elevation = 90.0 - zenith
    refraction_correction = calc_refraction(exoatm_elevation)
    solar_zen = zenith - refraction_correction
    elevation = 90.0 - solar_zen
    return {"azimuth": azimuth, "elevation": elevation}

def calc_sol_noon(jd, longitude, timezone):
    tnoon = calc_time_julian_cent(jd - longitude / 360.0)
    eq_time = calc_equation_of_time(tnoon)
    sol_noon_offset = 720.0 - (longitude * 4) - eq_time  # in minutes
    newt = calc_time_julian_cent(jd - 0.5 + sol_noon_offset / 1440.0)
    eq_time = calc_equation_of_time(newt)
    sol_noon_local = 720 - (longitude * 4) - eq_time + (timezone * 60.0)  # in minutes
    while sol_noon_local < 0.0:
        sol_noon_local += 1440.0
    while sol_noon_local >= 1440.0:
        sol_noon_local -= 1440.0
    return sol_noon_local

def calc_sunrise_set_utc(rise, JD, latitude, longitude):
    t = calc_time_julian_cent(JD)
    eq_time = calc_equation_of_time(t)
    solar_dec = calc_sun_declination(t)
    hour_angle = calc_hour_angle_sunrise(latitude, solar_dec)
    if not rise:
        hour_angle = -hour_angle
    delta = longitude + rad_to_deg(hour_angle)
    time_utc = 720 - (4.0 * delta) - eq_time  # in minutes
    return time_utc

def calc_sunrise_set(rise, JD, latitude, longitude, timezone):
    time_utc = calc_sunrise_set_utc(rise, JD, latitude, longitude)
    new_time_utc = calc_sunrise_set_utc(rise, JD + time_utc / 1440.0, latitude, longitude)
    if is_number(new_time_utc):
        time_local = new_time_utc + (timezone * 60.0)
        rise_t = calc_time_julian_cent(JD + new_time_utc / 1440.0)
        rise_az_el = calc_az_el(rise_t, time_local, latitude, longitude, timezone)
        azimuth = rise_az_el["azimuth"]
        jday = JD
        while time_local < 0.0 or time_local >= 1440.0:
            increment = 1 if time_local < 0 else -1
            time_local += increment * 1440.0
            jday -= increment
    else:
        azimuth = -1.0
        time_local = 0.0
        doy = calc_doy_from_jd(JD)
        if ((latitude > 66.4 and doy > 79 and doy < 267) or
            (latitude < -66.4 and (doy < 83 or doy > 263))):
            jday = calc_jd_of_next_prev_rise_set(not rise, rise, JD, latitude, longitude, timezone)
        else:
            jday = calc_jd_of_next_prev_rise_set(rise, rise, JD, latitude, longitude, timezone)
    return {"jday": jday, "timelocal": time_local, "azimuth": azimuth}

def calc_jd_of_next_prev_rise_set(next, rise, JD, latitude, longitude, tz):
    julianday = JD
    increment = 1.0 if next else -1.0
    time = calc_sunrise_set_utc(rise, julianday, latitude, longitude)
    while not is_number(time):
        julianday += increment
        time = calc_sunrise_set_utc(rise, julianday, latitude, longitude)
    time_local = time + tz * 60.0
    while time_local < 0.0 or time_local >= 1440.0:
        incr = 1 if time_local < 0 else -1
        time_local += incr * 1440.0
        julianday -= incr
    return julianday


In [2]:
from datetime import datetime, timedelta

year = 2024
month = 8
day = 4

# Example usage to calculate sunrise and sunset for Kona, Hawaii
latitude = 19.639994  # Latitude for Kona, Hawaii
longitude = -155.996926  # Longitude for Kona, Hawaii
timezone = -10  # Hawaii-Aleutian Standard Time

#latitude = 47.6062  # Latitude for Seattle, Washington
#longitude = -122.3321  # Longitude for Seattle, Washington
#timezone = -7  # Pacific Daylight Time

jd = get_jd(year, month, day)

sunrise = calc_sunrise_set(True, jd, latitude, longitude, timezone)
sunset = calc_sunrise_set(False, jd, latitude, longitude, timezone)

# Convert minutes since midnight to datetime
def convert_to_datetime(minutes, year, month, day):
    hours, minutes = divmod(minutes, 60)
    return datetime(year, month, day, int(hours), int(minutes))

sunrise_time = convert_to_datetime(sunrise['timelocal'], year, month, day)
sunset_time = convert_to_datetime(sunset['timelocal'], year, month, day)

print(f"Sunrise: {sunrise_time.strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Sunset: {sunset_time.strftime('%Y-%m-%d %H:%M:%S')}")


Sunrise: 2024-08-04 06:01:00
Sunset: 2024-08-04 18:58:00


In [3]:
sunrise

{'jday': 2460526.5,
 'timelocal': 361.2968332427238,
 'azimuth': 71.6208711457897}

In [4]:
sunset

{'jday': 2460526.5,
 'timelocal': 1138.5051888257854,
 'azimuth': 288.22222611007487}